# 02: Fetch Cochrane PDFs via Wiley TDM API

## Summary
This notebook downloads Cochrane review PDFs using the Wiley Text and Data Mining (TDM) API. The PDFs contain structured reference sections that properly categorize included vs. excluded studies - information not available in PubMed XML.

**Pipeline Position:** Third notebook - downloads source PDFs for extracting categorized references.

**What this notebook does:**
1. Loads Cochrane review metadata from PubMed (including DOIs)
2. Downloads PDFs for each review via Wiley TDM API
3. Implements **smart batching with automatic resume** after quota limits
4. Tracks download progress in persistent log for interruption recovery
5. Saves PDFs to local storage for text extraction

**Input:** `Data/cochrane_pubmed_abstracts.csv`

**Output:** 
- `Data/cochrane_pdfs/*.pdf` - Downloaded PDF files
- `Data/pdf_download_log.csv` - Persistent download tracking log

**Requirements:**
- Wiley TDM API token in `.env` file (WILEY_TEXT_AND_DATA_MINING_TOKEN)
- Institutional IP access to Cochrane Library content

**Important:** PDFs contain proprietary content and must NOT be uploaded to GitHub. They are excluded via .gitignore.

## Resumable Download Strategy
The download process is designed to handle API quota limits gracefully:
- **Persistent tracking:** Download status saved after each PDF
- **Smart resume:** Re-running skips already downloaded or permanently failed DOIs
- **Quota detection:** Automatically stops on rate limit and reports when to retry
- **Batch processing:** Downloads in configurable batches with progress checkpoints

In [87]:
# Install required packages for PDF download and processing
%pip install -q wiley-tdm python-dotenv pandas biopython

Note: you may need to restart the kernel to use updated packages.


In [88]:
# Set up environment and load credentials
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd

notebook_dir = Path.cwd()
project_root = notebook_dir if (notebook_dir / ".env").exists() else notebook_dir.parent
env_path = project_root / ".env"
load_dotenv(env_path, override=True)

WILEY_TDM_TOKEN = os.getenv("WILEY_TEXT_AND_DATA_MINING_TOKEN")
os.environ['TDM_API_TOKEN'] = WILEY_TDM_TOKEN or ""

DATA_DIR = project_root / "Data"
PDF_DIR = DATA_DIR / "cochrane_pdfs"
ABSTRACTS_CSV = DATA_DIR / "cochrane_pubmed_abstracts.csv"

print(f"Wiley TDM Token loaded: {'✓' if WILEY_TDM_TOKEN else '✗'}")
print(f"PDF output directory: {PDF_DIR}")

if not WILEY_TDM_TOKEN:
    raise ValueError("WILEY_TEXT_AND_DATA_MINING_TOKEN not set in .env file")

Wiley TDM Token loaded: ✓
PDF output directory: c:\Users\juanx\Documents\LSE-UKHSA Project\Data\cochrane_pdfs


In [89]:
# Load Cochrane reviews and fetch DOIs from PubMed
from Bio import Entrez
import time
import re

Entrez.email = os.getenv("NCBI_EMAIL", "")
Entrez.api_key = os.getenv("NCBI_API_KEY", "")

abstracts = pd.read_csv(ABSTRACTS_CSV, dtype={"pmid": str})
print(f"Loaded {len(abstracts):,} Cochrane reviews")

def get_doi_from_pmid(pmid):
    """Fetch DOI for a PubMed ID."""
    try:
        handle = Entrez.efetch(db="pubmed", id=pmid, rettype="xml", retmode="xml")
        xml = handle.read()
        if isinstance(xml, bytes):
            xml = xml.decode('utf-8')
        handle.close()
        doi_match = re.search(r'<ArticleId IdType="doi">([^<]+)</ArticleId>', xml)
        return doi_match.group(1) if doi_match else None
    except Exception as e:
        return None

# Test with a sample
sample_pmid = abstracts['pmid'].iloc[0]
sample_doi = get_doi_from_pmid(sample_pmid)
print(f"Sample: PMID {sample_pmid} -> DOI {sample_doi}")

Loaded 17,298 Cochrane reviews
Sample: PMID 17636697 -> DOI 10.1002/14651858.CD002182


In [90]:
# Initialize Wiley TDM client and test download
from wiley_tdm import TDMClient

PDF_DIR.mkdir(parents=True, exist_ok=True)
tdm = TDMClient(download_dir=str(PDF_DIR))

# Test with a single download
if sample_doi:
    print(f"Testing download for DOI: {sample_doi}")
    result = tdm.download_pdf(sample_doi)
    print(f"Result: {result}")
else:
    print("Could not get DOI for test download")

Testing download for DOI: 10.1002/14651858.CD002182
Result: Existing File


In [91]:
# Smart download tracking - persistent log for resume capability
DOWNLOAD_LOG = DATA_DIR / "pdf_download_log.csv"
DOI_CACHE_FILE = DATA_DIR / "doi_cache.csv"

# Filter to reviews with valid DOIs (ALL years, ALL types - no filtering!)
valid_dois = abstracts[abstracts['doi'].notna() & (abstracts['doi'] != '')].copy()
valid_dois['year'] = pd.to_numeric(valid_dois['year'], errors='coerce')

# DEDUPLICATE by DOI - same DOI appears multiple times in PubMed data
# Keep the most recent version of each review
valid_dois = valid_dois.sort_values('year', ascending=False).drop_duplicates(subset=['doi'], keep='first')

print(f"Total UNIQUE Cochrane entries with valid DOIs: {len(valid_dois):,}")
print(f"Year range: {valid_dois['year'].min():.0f} - {valid_dois['year'].max():.0f}")

# Helper to convert DOI to safe filename
def doi_to_filename(doi):
    return doi.replace("/", "-").replace(":", "_")

valid_dois['filename'] = valid_dois['doi'].apply(doi_to_filename)

# Check which PDFs already exist on disk
existing_pdfs = set(f.stem for f in PDF_DIR.glob("*.pdf"))
print(f"PDFs already downloaded: {len(existing_pdfs):,}")

# Load existing download log to track failures and avoid re-attempting permanent failures
if DOWNLOAD_LOG.exists():
    prior_log = pd.read_csv(DOWNLOAD_LOG, dtype={'pmid': str, 'doi': str})
    # Consider these as permanent failures (don't retry)
    permanent_failures = prior_log[prior_log['status'].isin(['http_403', 'http_404', 'not_wiley'])]['doi'].tolist()
    # Quota failures should be retried
    quota_failures = prior_log[prior_log['status'].str.contains('http_500|quota', case=False, na=False)]['doi'].tolist()
    print(f"Prior log: {len(prior_log)} entries, {len(permanent_failures)} permanent failures, {len(quota_failures)} quota failures (will retry)")
else:
    prior_log = pd.DataFrame()
    permanent_failures = []
    quota_failures = []
    print("No prior download log found - starting fresh")

# Determine what needs downloading:
# - Not already on disk
# - Not a permanent failure (403, 404, non-Wiley)
to_download = valid_dois[
    (~valid_dois['filename'].isin(existing_pdfs)) & 
    (~valid_dois['doi'].isin(permanent_failures))
].copy()

# Sort by year (newest first - often more available)
to_download = to_download.sort_values('year', ascending=False)
print(f"\nRemaining to download: {len(to_download):,}")
print(f"Year distribution of remaining:")
print(to_download.groupby('year').size().sort_index(ascending=False).head(10))

Total UNIQUE Cochrane entries with valid DOIs: 16,616
Year range: 1996 - 2026
PDFs already downloaded: 15,277
Prior log: 15160 entries, 16 permanent failures, 2 quota failures (will retry)

Remaining to download: 1,323
Year distribution of remaining:
year
2026     18
2025    129
2024     97
2023    116
2022     12
2021     11
2020     17
2019     80
2018     50
2017     55
dtype: int64


In [92]:
# Resumable PDF downloader with ROBUST rate limiting and auto-retry
import requests
from tqdm.notebook import tqdm
from datetime import datetime, timedelta, timezone
from collections import deque
import json

# =============================================================================
# WILEY API RATE LIMITS (confirmed by Wiley support):
#   1) Up to 3 articles per second
#   2) Up to 60 requests per 10 minutes
#
# Strategy: Download quickly (respecting 3/sec), then pause when hitting 60/10min
# Expected rate: ~360 PDFs/hour (60 every 10 minutes)
# =============================================================================

# Configuration
REQUESTS_PER_10_MIN = 60  # Wiley's limit
WINDOW_SECONDS = 600  # 10 minutes in seconds
MIN_DELAY_BETWEEN_REQUESTS = 0.4  # Slightly under 3/sec to be safe
MAX_RETRIES = 5  # Retries for connection issues
INITIAL_BACKOFF = 30  # Initial wait on failure (seconds)
MAX_BACKOFF = 600  # Maximum wait between retries (10 minutes)
BATCH_SIZE = 50  # Save progress every N downloads

# Track request timestamps for rate limiting
request_timestamps = deque(maxlen=REQUESTS_PER_10_MIN)

def doi_to_safe_filename(doi):
    """Convert DOI to safe filename - MUST match the check in cell above."""
    return doi.replace("/", "-").replace(":", "_")

def wait_for_rate_limit():
    """Wait if we've hit the 60 requests / 10 minutes limit."""
    now = time.time()
    
    # Clean old timestamps (older than 10 minutes)
    while request_timestamps and (now - request_timestamps[0]) > WINDOW_SECONDS:
        request_timestamps.popleft()
    
    # If we've made 60 requests in the last 10 minutes, wait for oldest to expire
    if len(request_timestamps) >= REQUESTS_PER_10_MIN:
        oldest = request_timestamps[0]
        wait_time = WINDOW_SECONDS - (now - oldest) + 2  # Add 2s buffer
        if wait_time > 0:
            print(f"\n⏳ Hit 60/10min limit - waiting {wait_time:.0f}s for window to slide...")
            time.sleep(wait_time)
    
    # Maintain minimum delay between requests (3/sec limit)
    if request_timestamps:
        time_since_last = time.time() - request_timestamps[-1]
        if time_since_last < MIN_DELAY_BETWEEN_REQUESTS:
            time.sleep(MIN_DELAY_BETWEEN_REQUESTS - time_since_last)

def download_pdf_with_retry(doi, token, output_dir, max_retries=MAX_RETRIES):
    """Download PDF with exponential backoff retry logic."""
    url = f"https://api.wiley.com/onlinelibrary/tdm/v1/articles/{doi}"
    headers = {"Wiley-TDM-Client-Token": token}
    
    for attempt in range(max_retries + 1):
        # Respect rate limit before making request
        wait_for_rate_limit()
        
        try:
            # Record this request timestamp
            request_timestamps.append(time.time())
            
            response = requests.get(url, headers=headers, timeout=60)
            
            if response.status_code == 200:
                content_type = response.headers.get('Content-Type', '')
                if 'application/pdf' in content_type and len(response.content) > 1000:
                    filename = doi_to_safe_filename(doi) + ".pdf"
                    filepath = output_dir / filename
                    with open(filepath, 'wb') as f:
                        f.write(response.content)
                    return "success", len(response.content)
                elif 'application/json' in content_type:
                    try:
                        error_data = response.json()
                        if 'quota' in str(error_data).lower():
                            return "quota_exceeded", 0
                    except:
                        pass
                    return "api_error", 0
                else:
                    return "empty_response", 0
                    
            elif response.status_code == 500:
                # Check if it's a quota violation
                try:
                    if 'application/json' in response.headers.get('Content-Type', ''):
                        error_data = response.json()
                        if 'quota' in str(error_data).lower() or 'ratelimit' in str(error_data).lower():
                            return "quota_exceeded", 0
                except:
                    pass
                # HTTP 500 might be temporary - retry with backoff
                if attempt < max_retries:
                    backoff = min(INITIAL_BACKOFF * (2 ** attempt), MAX_BACKOFF)
                    print(f"\n⚠️  HTTP 500 for {doi[:30]}... - retrying in {backoff}s (attempt {attempt+1}/{max_retries})")
                    time.sleep(backoff)
                    continue
                return "http_500", 0
                
            elif response.status_code in [403, 404]:
                # Permanent failures - don't retry
                return f"http_{response.status_code}", 0
                
            elif response.status_code in [429, 503]:
                # Rate limited or service unavailable - wait and retry
                if attempt < max_retries:
                    backoff = min(INITIAL_BACKOFF * (2 ** attempt), MAX_BACKOFF)
                    print(f"\n⚠️  HTTP {response.status_code} - server busy, waiting {backoff}s...")
                    time.sleep(backoff)
                    continue
                return f"http_{response.status_code}", 0
            else:
                return f"http_{response.status_code}", 0
                
        except requests.Timeout:
            if attempt < max_retries:
                backoff = min(INITIAL_BACKOFF * (2 ** attempt), MAX_BACKOFF)
                print(f"\n⚠️  Timeout for {doi[:30]}... - retrying in {backoff}s")
                time.sleep(backoff)
                continue
            return "timeout", 0
            
        except requests.ConnectionError:
            if attempt < max_retries:
                backoff = min(INITIAL_BACKOFF * (2 ** attempt), MAX_BACKOFF)
                print(f"\n🔌 Connection error - retrying in {backoff}s (attempt {attempt+1}/{max_retries})")
                time.sleep(backoff)
                continue
            return "connection_error", 0
            
        except Exception as e:
            if attempt < max_retries:
                backoff = min(INITIAL_BACKOFF * (2 ** attempt), MAX_BACKOFF)
                print(f"\n⚠️  Error {type(e).__name__} - retrying in {backoff}s")
                time.sleep(backoff)
                continue
            return f"error_{type(e).__name__}", 0
    
    return "max_retries_exceeded", 0

def save_progress(results_log, log_path):
    """Save current progress to CSV."""
    if results_log:
        log_df = pd.DataFrame(results_log)
        if log_path.exists():
            existing = pd.read_csv(log_path, dtype={'pmid': str, 'doi': str})
            combined = pd.concat([existing, log_df]).drop_duplicates(subset=['doi'], keep='last')
            combined.to_csv(log_path, index=False)
        else:
            log_df.to_csv(log_path, index=False)

# =============================================================================
# MAIN DOWNLOAD LOOP - Runs until complete, handles all errors gracefully
# =============================================================================

print(f"🚀 Starting ROBUST download of {len(to_download):,} PDFs")
print(f"   Rate limits: 3/sec AND 60/10min → ~360 PDFs/hour max")
print(f"   Retries: {MAX_RETRIES} attempts with exponential backoff")
print(f"   Estimated time: ~{len(to_download)/360:.1f} hours")
print("-" * 60)

success_count = 0
fail_count = 0
results_log = []
total_bytes = 0
start_time = datetime.now()
rate_limit_waits = 0

i = 0
pbar = tqdm(total=len(to_download), desc="Downloading", unit="pdf")

while i < len(to_download):
    row = to_download.iloc[i]
    doi = row['doi']
    pmid = row['pmid']
    
    status, file_size = download_pdf_with_retry(doi, WILEY_TDM_TOKEN, PDF_DIR)
    
    if status == "success":
        success_count += 1
        total_bytes += file_size
        pbar.update(1)
        i += 1
        
    elif status == "quota_exceeded":
        # Unexpected quota hit - wait 10 minutes and retry
        rate_limit_waits += 1
        print(f"\n⏰ QUOTA EXCEEDED at {datetime.now().strftime('%H:%M:%S')}")
        print(f"   Progress so far: {success_count} downloaded, {fail_count} failed")
        print(f"   Waiting 10 minutes for quota reset...")
        
        # Save progress before waiting
        save_progress(results_log, DOWNLOAD_LOG)
        results_log = []
        
        # Clear our tracking (server's window may differ)
        request_timestamps.clear()
        
        # Wait with countdown
        for remaining in range(600, 0, -60):
            print(f"   ⏳ {remaining//60} minutes remaining...", end='\r')
            time.sleep(min(60, remaining))
        print(f"\n✓ Resuming downloads...")
        # Don't increment i - retry the same DOI
        continue
        
    else:
        # Permanent failure (403, 404, etc.) - log and move on
        fail_count += 1
        results_log.append({
            'pmid': pmid,
            'doi': doi,
            'year': row['year'],
            'status': status,
            'size_bytes': file_size,
            'timestamp': datetime.now().isoformat()
        })
        pbar.update(1)
        i += 1
    
    # Log successful downloads too
    if status == "success":
        results_log.append({
            'pmid': pmid,
            'doi': doi,
            'year': row['year'],
            'status': status,
            'size_bytes': file_size,
            'timestamp': datetime.now().isoformat()
        })
    
    # Save progress periodically
    if len(results_log) >= BATCH_SIZE:
        save_progress(results_log, DOWNLOAD_LOG)
        elapsed = (datetime.now() - start_time).total_seconds() / 3600
        rate = success_count / elapsed if elapsed > 0 else 0
        remaining_time = (len(to_download) - i) / rate if rate > 0 else 0
        print(f"\n   📊 {success_count} done, {fail_count} failed | {rate:.0f}/hr | ~{remaining_time:.1f}hr remaining")
        results_log = []

pbar.close()

# Final save
save_progress(results_log, DOWNLOAD_LOG)

# =============================================================================
# SESSION SUMMARY
# =============================================================================
elapsed_hours = (datetime.now() - start_time).total_seconds() / 3600
print("\n" + "=" * 60)
print("✅ DOWNLOAD SESSION COMPLETE")
print("=" * 60)
print(f"Duration: {elapsed_hours:.1f} hours")
print(f"Downloaded: {success_count:,} PDFs ({total_bytes/1024/1024:.1f} MB)")
print(f"Failed (permanent): {fail_count:,}")
print(f"Rate limit pauses: {rate_limit_waits}")
print(f"Actual rate: {success_count/elapsed_hours:.0f} PDFs/hour" if elapsed_hours > 0 else "")
print(f"\n📁 PDFs saved to: {PDF_DIR}")

🚀 Starting ROBUST download of 1,323 PDFs
   Rate limits: 3/sec AND 60/10min → ~360 PDFs/hour max
   Retries: 5 attempts with exponential backoff
   Estimated time: ~3.7 hours
------------------------------------------------------------


Downloading:   0%|          | 0/1323 [00:00<?, ?pdf/s]


   📊 50 done, 0 failed | 1958/hr | ~0.7hr remaining

⏳ Hit 60/10min limit - waiting 491s for window to slide...

⏰ QUOTA EXCEEDED at 11:02:04
   Progress so far: 60 downloaded, 0 failed
   Waiting 10 minutes for quota reset...
   ⏳ 1 minutes remaining....
✓ Resuming downloads...

   📊 110 done, 0 failed | 307/hr | ~4.0hr remaining

⏳ Hit 60/10min limit - waiting 496s for window to slide...

⏰ QUOTA EXCEEDED at 11:22:06
   Progress so far: 120 downloaded, 0 failed
   Waiting 10 minutes for quota reset...
   ⏳ 1 minutes remaining....
✓ Resuming downloads...

   📊 170 done, 0 failed | 245/hr | ~4.7hr remaining

⏳ Hit 60/10min limit - waiting 496s for window to slide...

⏰ QUOTA EXCEEDED at 11:42:09
   Progress so far: 180 downloaded, 0 failed
   Waiting 10 minutes for quota reset...
   ⏳ 1 minutes remaining....
✓ Resuming downloads...

   📊 230 done, 0 failed | 224/hr | ~4.9hr remaining

⏳ Hit 60/10min limit - waiting 497s for window to slide...

   📊 280 done, 0 failed | 234/hr | ~4.5hr

In [93]:
# Overall download progress summary
pdf_files = list(PDF_DIR.glob("*.pdf"))
total_size_mb = sum(p.stat().st_size for p in pdf_files) / (1024 * 1024)

# Load full log for analysis
if DOWNLOAD_LOG.exists():
    full_log = pd.read_csv(DOWNLOAD_LOG, dtype={'pmid': str, 'doi': str})
    status_summary = full_log['status'].value_counts()
else:
    full_log = pd.DataFrame()
    status_summary = pd.Series(dtype=int)

print("=" * 60)
print("OVERALL DOWNLOAD PROGRESS")
print("=" * 60)
print(f"Total Cochrane reviews with DOIs: {len(valid_dois):,}")
print(f"PDFs on disk: {len(pdf_files):,}")
print(f"Storage used: {total_size_mb:.1f} MB ({total_size_mb/1024:.2f} GB)")
print(f"Average PDF size: {total_size_mb/len(pdf_files)*1024:.0f} KB" if pdf_files else "")
print(f"\nProgress: {len(pdf_files)/len(valid_dois)*100:.1f}% complete")

# Remaining work
remaining = len(valid_dois) - len(pdf_files) - len(full_log[full_log['status'].isin(['http_403', 'http_404'])])
print(f"Remaining downloadable: ~{remaining:,}")

if not status_summary.empty:
    print(f"\nDownload status breakdown:")
    for status, count in status_summary.items():
        print(f"  {status}: {count:,}")

print(f"\n📋 Next steps:")
if len(pdf_files) >= 100:
    print(f"   ✓ Enough PDFs to validate pipeline - proceed to notebook 03")
if remaining > 0:
    print(f"   → Re-run download cell after quota reset to continue")
else:
    print(f"   ✓ All available PDFs downloaded!")

OVERALL DOWNLOAD PROGRESS
Total Cochrane reviews with DOIs: 16,616
PDFs on disk: 16,588
Storage used: 12890.0 MB (12.59 GB)
Average PDF size: 796 KB

Progress: 99.8% complete
Remaining downloadable: ~0

Download status breakdown:
  success: 16,359
  failed: 94
  http_404: 28
  quota_exceeded: 2

📋 Next steps:
   ✓ Enough PDFs to validate pipeline - proceed to notebook 03
   ✓ All available PDFs downloaded!
